# Chess Tactical Analysis

This notebook provides detailed tactical analysis of your chess games, including:
- Blunder and mistake detection
- Tactical pattern recognition
- Accuracy calculations
- Personalized improvement recommendations

## Setup

In [16]:
# Import libraries
import sys
import os
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import plotly.express as px
import plotly.graph_objects as go
from datetime import datetime, timedelta

# Add src directory to path
sys.path.append('../src')
sys.path.append('../config')
sys.path.append('..')

# Import chess analysis modules
from src.data_fetcher import ChessComDataFetcher
from src.game_parser import GameParser
from src.analyzers.tactical_analyzer import TacticalAnalyzer
from config.settings import Config

print("✅ Libraries imported successfully!")

✅ Libraries imported successfully!


## Configuration

Set up your Chess.com username and analysis parameters.

In [17]:
# Configuration
USERNAME = Config.CHESS_COM_USERNAME or "vds4321"  # Your Chess.com username
ANALYSIS_MONTHS = 1  # Number of months to analyze
TIME_CONTROLS = ["blitz", "rapid"]  # Focus on longer time controls for tactical analysis

print(f"✅ Username: {USERNAME}")
print(f"📊 Analysis period: {ANALYSIS_MONTHS} months")
print(f"⏱️  Time controls: {', '.join(TIME_CONTROLS)}")

✅ Username: vds4321
📊 Analysis period: 1 months
⏱️  Time controls: blitz, rapid


## Download and Parse Games

In [18]:
# Initialize data fetcher and parser
fetcher = ChessComDataFetcher(USERNAME)
parser = GameParser()

print("📥 Downloading recent games...")

# Get recent games
end_date = datetime.now()
start_date = end_date - timedelta(days=30 * ANALYSIS_MONTHS)

raw_games = fetcher.get_all_games(
    start_date=start_date,
    end_date=end_date
)

# Filter by time controls
filtered_games = []
for game in raw_games:
    time_class = game.get('time_class', '').lower()
    if time_class in [tc.lower() for tc in TIME_CONTROLS]:
        filtered_games.append(game)

print(f"✅ Downloaded {len(raw_games)} total games")
print(f"✅ Filtered to {len(filtered_games)} games matching time controls")

# Parse games for tactical analysis
print("🔍 Parsing games with Stockfish analysis...")
parsed_games = []

# Parse a sample of games (limit for performance)
sample_games = filtered_games[:20]  # Analyze up to 20 recent games

for i, game in enumerate(sample_games):
    try:
        parsed_game = parser.parse_chess_com_game(game)
        if parsed_game:
            parsed_games.append(parsed_game)
        print(f"Parsed game {i+1}/{len(sample_games)}", end='\r')
    except Exception as e:
        print(f"Error parsing game {i+1}: {e}")

print(f"\n✅ Successfully parsed {len(parsed_games)} games for tactical analysis")

INFO:src.game_parser:Stockfish engine initialized successfully
INFO:src.data_fetcher:Fetching games for vds4321
INFO:src.data_fetcher:Using cached data for archives_vds4321


📥 Downloading recent games...


Fetching game archives: 100%|███████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 1/1 [00:00<00:00,  9.37it/s]
INFO:src.data_fetcher:Fetched 62 games


✅ Downloaded 62 total games
✅ Filtered to 62 games matching time controls
🔍 Parsing games with Stockfish analysis...
Parsed game 20/20
✅ Successfully parsed 20 games for tactical analysis


## Tactical Analysis

In [19]:
# Initialize tactical analyzer
tactical_analyzer = TacticalAnalyzer()

print("🎯 Running comprehensive tactical analysis...")

# Run tactical analysis on parsed games
tactical_results = tactical_analyzer.analyze_tactical_patterns(parsed_games)

print("\n📊 Tactical Analysis Results:")
print("=" * 40)
print(f"Games analyzed: {tactical_results.get('total_games', 0)}")
print(f"Average accuracy: {tactical_results.get('average_accuracy', 0):.1f}%")
print(f"Total moves: {tactical_results.get('total_moves', 0)}")
print(f"Blunders: {tactical_results.get('total_blunders', 0)}")
print(f"Mistakes: {tactical_results.get('total_mistakes', 0)}")
print(f"Inaccuracies: {tactical_results.get('total_inaccuracies', 0)}")

# Calculate blunder rate
if tactical_results.get('total_moves', 0) > 0:
    blunder_rate = (tactical_results.get('total_blunders', 0) / tactical_results.get('total_moves', 0)) * 100
    print(f"Blunder rate: {blunder_rate:.2f}%")

🎯 Running comprehensive tactical analysis...

📊 Tactical Analysis Results:
Games analyzed: 0
Average accuracy: 0.0%
Total moves: 0
Blunders: 0
Mistakes: 0
Inaccuracies: 0


## Visualizations

In [ ]:
# Create tactical accuracy chart
if 'game_data' in tactical_results:
    game_data = tactical_results['game_data']
    
    # Extract game accuracies
    accuracies = [game.get('accuracy', 0) for game in game_data]
    game_numbers = list(range(1, len(accuracies) + 1))
    
    # Create accuracy trend plot
    fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(15, 6))
    
    # Accuracy over time
    ax1.plot(game_numbers, accuracies, marker='o', linewidth=2, markersize=6)
    ax1.axhline(y=np.mean(accuracies), color='red', linestyle='--', alpha=0.7, label=f'Average: {np.mean(accuracies):.1f}%')
    ax1.set_xlabel('Game Number')
    ax1.set_ylabel('Accuracy (%)')
    ax1.set_title('Tactical Accuracy Over Recent Games')
    ax1.legend()
    ax1.grid(True, alpha=0.3)
    
    # Accuracy distribution
    ax2.hist(accuracies, bins=10, edgecolor='black', alpha=0.7)
    ax2.axvline(x=np.mean(accuracies), color='red', linestyle='--', alpha=0.7, label=f'Average: {np.mean(accuracies):.1f}%')
    ax2.set_xlabel('Accuracy (%)')
    ax2.set_ylabel('Number of Games')
    ax2.set_title('Accuracy Distribution')
    ax2.legend()
    
    plt.tight_layout()
    plt.show()
    
    print(f"📈 Average accuracy: {np.mean(accuracies):.1f}%")
    print(f"📊 Best game accuracy: {max(accuracies):.1f}%")
    print(f"📉 Lowest game accuracy: {min(accuracies):.1f}%")

## Tactical Recommendations

In [ ]:
# Get personalized tactical recommendations
recommendations = tactical_analyzer.get_tactical_recommendations(tactical_results)

print("🎯 Personalized Tactical Improvement Recommendations:")
print("=" * 55)

for i, recommendation in enumerate(recommendations, 1):
    print(f"{i}. {recommendation}")
    print()

## Detailed Game Analysis

Analyze individual games for specific tactical patterns.

In [ ]:
# Analyze the most recent game in detail
if parsed_games:
    recent_game = parsed_games[0]
    
    print(f"🔍 Detailed analysis of recent game:")
    print(f"   White: {recent_game.get('white_username', 'Unknown')}")
    print(f"   Black: {recent_game.get('black_username', 'Unknown')}")
    print(f"   Result: {recent_game.get('result', 'Unknown')}")
    print(f"   Time Control: {recent_game.get('time_control', 'Unknown')}")
    
    # Get the PGN for tactical analysis
    if 'pgn' in recent_game:
        game_tactics = tactical_analyzer.analyze_game_tactics(recent_game['pgn'], USERNAME)
        
        print(f"\n📊 Game tactical summary:")
        print(f"   Accuracy: {game_tactics.get('accuracy', 0):.1f}%")
        print(f"   Blunders: {game_tactics.get('blunders', 0)}")
        print(f"   Mistakes: {game_tactics.get('mistakes', 0)}")
        print(f"   Inaccuracies: {game_tactics.get('inaccuracies', 0)}")
    else:
        print("   ⚠️ No PGN data available for detailed analysis")
else:
    print("❌ No games available for detailed analysis")

## Summary

Your tactical analysis is complete! Use the insights above to focus your training on specific areas of improvement.